In [1]:
import sys
!{sys.executable} -m pip install graphviz

!{sys.executable} -m pip install isoweek
!{sys.executable} -m pip install pandas_summary
!{sys.executable} -m pip install ipython

In [4]:
!{sys.executable} -m pip install sklearn-pandas

  Using cached sklearn_pandas-1.8.0-py2.py3-none-any.whl (12 kB)


In [2]:
!{sys.executable} -m pip uninstall -y sklearn-pandas

In [40]:
!pip list | grep scikit-learn

scikit-learn                       0.23.1


In [41]:
!{sys.executable} -m pip uninstall -y scikit-learn
!{sys.executable} -m pip install scikit-learn==0.20.4

Found existing installation: scikit-learn 0.23.1
Uninstalling scikit-learn-0.23.1:
  Successfully uninstalled scikit-learn-0.23.1
     |████████████████████████████████| 5.4 MB 5.3 MB/s eta 0:00:01


In [42]:
!pip list | grep scikit-learn

scikit-learn                       0.20.4


In [ ]:
!pip list | grep pandas

In [ ]:

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [43]:
from fastai_custom.imports import *
from fastai_custom.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn.metrics import accuracy_score

from sklearn import metrics
set_plot_sizes(12,14,16)

In [7]:
!{sys.executable} -m pip list | grep scikit-learn

scikit-learn                       0.23.1


In [8]:
!{sys.executable} -m pip list | grep sklearn-pandas

sklearn-pandas                     1.8.0


In [ ]:
!gsutil cp -r gs://sk-kfp-kubeflowpipelines-eu/data/ .

In [ ]:
pwd

In [44]:
import os
PATH=os.path.join(os.getcwd(),'data')

In [ ]:
!ls {PATH}

In [45]:
df_raw = pd.read_csv(f'{PATH}/training/dataset.csv', low_memory=False)

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(df_raw.tail().T)

In [ ]:
display_all(df_raw.describe(include='all').T)

In [ ]:
#m = RandomForestClassifier(n_jobs=-1)
#m.fit(df_raw.drop('Cover_Type', axis=1), df_raw.Cover_Type)

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.

## Test data - converting strings to categories

Same categories need to be applied to validation/testing data using ***apply_cats()***

In [46]:
train_cats(df_raw)

In [ ]:
df_raw.Wilderness_Area.cat.categories

In [ ]:
df_raw.Wilderness_Area.cat.codes

## Test time - supply the nas created by proc_df

which columns are missing values, what are the median values of those columns

In [ ]:

display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [ ]:
len(df_raw)

In [47]:
df_test = pd.read_csv(f'{PATH}/validation/dataset.csv', low_memory=False)

In [ ]:
len(df_test)

In [48]:
df, y, nas = proc_df(df_raw, 'Cover_Type')

In [49]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 9836  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((30173, 12), (30173,), (9836, 12))

In [50]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [51]:
m = RandomForestClassifier(n_estimators=200, n_jobs=-1,oob_score=True,min_samples_leaf=1,max_features='sqrt')
m.fit(X_train, y_train)
print_score(m)

[1.0, 0.7817202114680765, 0.8708116528021741]


In [ ]:
import pickle

In [ ]:
os.remove('model.pkl')
pickle.dump(m, open('model.pkl', 'wb'))

In [ ]:
loaded_model = pickle.load(open('model.pkl', 'rb'))
print_score(loaded_model)

In [ ]:
m

In [52]:
os.remove('model.joblib')
from joblib import dump

dump(m, 'model.joblib')

['model.joblib']

In [ ]:
!gsutil cp model.joblib 

### pickle dataframe with categories for ***apply_cats()***


In [ ]:
df_raw = df_raw[0:0]

In [ ]:
df_raw.Wilderness_Area.cat.categories

In [ ]:
df_raw.Wilderness_Area.cat.codes

In [ ]:
df_raw.to_pickle("./df_raw.pkl")

In [ ]:
unpickled_df_raw = pd.read_pickle("./df_raw.pkl")

In [ ]:
unpickled_df_raw.Wilderness_Area.cat.categories

In [ ]:
apply_cats(df_test,unpickled_df_raw)

In [ ]:
df_test.Wilderness_Area.cat.categories

In [ ]:
df_test.Wilderness_Area.cat.codes

### Preprocess test dataset

TODO - serialize nas if missing values\
TODO - serialize mappers if standardizing/normalizing

In [ ]:
df_pred, y_pred, nas = proc_df(df_test, 'Cover_Type')

In [ ]:
m.score(df_pred,y_pred)

In [ ]:
m.predict(df_pred).tolist()[0:10]

In [ ]:
df_pred.loc[0]

### Test serialization of preprocessor

In [34]:
from preprocess import TrainedCategories

In [35]:
t = TrainedCategories(df_raw)

In [36]:
pickle.dump(t, open('preprocessor.pkl', 'wb'))

In [37]:
!gsutil cp ./preprocessor.pkl gs://my-custom-code

Copying file://./preprocessor.pkl [Content-Type=application/octet-stream]...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [ ]:
loaded_TrainedCategories = pickle.load(open('preprocessor.pkl', 'rb'))

In [ ]:
loaded_df=loaded_TrainedCategories.preprocess()

In [ ]:
loaded_df.Wilderness_Area.cat.categories

In [ ]:
#os.remove('preprocess.pkl')

### Test create dataframe from column names and list

In [ ]:
cols = loaded_df.columns
cols = pd.Index.drop(cols,'Cover_Type')

In [ ]:
lst = [1979.0, 4.0, 38.0, 30.0, 19.0, 90.0, 146.0, 142.0, 113.0, 212.0, "Cache", "C4703"]

In [ ]:
df_re = pd.DataFrame(lst,cols)

In [ ]:
df_re

### test predictor.py

In [ ]:
from predictor import MyPredictor

In [ ]:
os.getcwd()

In [ ]:
MyPredictor.from_path(os.getcwd())

In [ ]:
predictor = MyPredictor.from_path(os.getcwd())

In [ ]:
lst

In [ ]:
test_instances = [1979.0, 4.0, 38.0, 30.0, 19.0, 90.0, 146.0, 142.0, 113.0, 212.0, 'Cache', 'C4703']
test_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type']
pd.DataFrame([test_instances],columns=test_cols)

In [ ]:
predictor.predict(lst)

In [ ]:
### Test packaging

In [55]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
writing my_custom_code.egg-info/PKG-INFO
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing requirements to my_custom_code.egg-info/requires.txt
writing top-level names to my_custom_code.egg-info/top_level.txt
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-2.1
creating my_custom_code-2.1/fastai_custom
creating my_custom_code-2.1/my_custom_code.egg-info
copying files to my_custom_code-2.1...
copying MANIFEST.in -> my_custom_code-2.1
copying predictor.py -> my_custom_code-2.1
copying preprocess.py -> my_custom_code-2.1
copying requirements.txt -> my_custom_code-2.1
copying setup.py -> my_custom_code-2.1
copying fastai_custom/__init__.py -> my_custom_code-2.1/fastai_custom
copying fastai_custom/imports.py -> my_custom_code-2.1/fastai_custom
copying fastai_custom/structured.py -

In [ ]:
!tar -tvf ./dist/my_custom_code-0.1.tar.gz

In [ ]:
!pip install ./dist/my_custom_code-1.0.tar.gz

### Upload source dist containing ***predictor.py*** and ***preprocess.py*** to GCS

In [56]:
!gsutil cp dist/my_custom_code-2.1.tar.gz gs://my-custom-code/staging-dir/

Copying file://dist/my_custom_code-2.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  8.7 KiB/  8.7 KiB]                                                
Operation completed over 1 objects/8.7 KiB.                                      


In [ ]:
!gsutil cp requirements.txt gs://my-custom-code/staging-dir/

### Create AI Platform ***model***

In [ ]:
!gcloud ai-platform models create fastai_model \
  --regions europe-west1

In [28]:

!PROJECT_ID='sk-kfp'
!BUCKET_NAME="my-custom-code"

!REGION='europe-west1'

### copy MODEL to GCS

In [53]:
!gsutil cp ./model.joblib gs://my-custom-code/model.joblib

Copying file://./model.joblib [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][177.6 MiB/177.6 MiB]                                                
Operation completed over 1 objects/177.6 MiB.                                    


In [ ]:
!apt-get install google-cloud-sdk

In [ ]:
MODEL_NAME='fastai_model'
MODEL_DIR='gs://my-custom-code/'
CUSTOM_CODE_PATH='gs://my-custom-code/staging-dir/my_custom_code-2.1.tar.gz'
PREDICTOR_CLASS='predictor.MyPredictor'
VERSION_NAME='v2_1'
#gcloud components install beta

gcloud beta ai-platform versions create $VERSION_NAME \
  --model=$MODEL_NAME \
  --origin=$MODEL_DIR \
  --runtime-version=1.15 \
  --python-version=3.7 \
  --package-uris=$CUSTOM_CODE_PATH \
  --prediction-class=$PREDICTOR_CLASS

In [ ]:
!pip list > requirements_test.txt

In [54]:
required=''

with open('requirement_works.txt') as f:
    required = f.read().splitlines()
    del required[0:2]
    req_list = []
    for item in required:
        req_list.append(item.split(' ')[0]+'=='+item.split(' ')[-1])
        #req_list.append(item.split(' ')[0])
#     req_list.remove('anaconda-project')
#     req_list.remove('anaconda-client')
#     req_list.remove('conda-package-handling')
#     req_list.remove('conda')
#     req_list.remove('jupyter-aihub-deploy-extension')
#     req_list.remove('mkl-fft')
    with open('requirements.txt','w') as f:
        f.write("\n".join(req_list))
         

In [ ]:
!pip freeze

In [ ]:
!pip list | grep sklearn